In [1]:
import pandas as pd
from prettytable import PrettyTable
from numpy import random
POPULATION_SIZE= 10
TOP=2
### DATA CELL ###
#import data from the same folder

import_student = pd.read_csv (r'studentNames.csv')
import_teacher = pd.read_csv (r'teachers.csv')
import_course = pd.read_csv (r'courses.csv')
import_student_name_course = pd.read_csv (r'studentCourse.csv')

student_names=import_student['A']
teacher_names=import_teacher['A']
courses=import_course
student_course=import_student_name_course
#print(student_names)
#print("\n\n")
#print(teacher_names)
#print("\n\n")
#print(courses)
#print("\n\n")
#print(student_course)

ROOMS=['C301','C302','C303','C304','C305','C306','C307','C308','C309','C3010']
SLOTS=['M1(9-12)','M1(2-5)','T1(9-12)','T1(2-5)','W1(9-12)','W1(2-5)','TH1(9-12)','TH1(2-5)','F1(9-12)','F1(2-5)','S1(9-12)','S1(2-5)','SN1(9-12)','SN1(2-5)','M2(9-12)','M2(2-5)','T2(9-12)','T2(2-5)','W2(9-12)','W2(2-5)','TH2(9-12)','TH2(2-5)','F2(9-12)','F2(2-5)']

In [2]:
from collections import defaultdict
import random as rnd

student_dict=defaultdict(list)
for i in range(len(student_course)):
    student_dict[student_course['StudentName'][i]].append(student_course['CourseCode'][i])


class Exams:
    def __init__(self, course):
        self.room=""
        self.time=""
        self.course=course
        self.teacher=""

class TimeTable:
    def __init__(self):
        self.exams=[]
        self.conflicts=0
        self.fitness=-1
    def initialize(self):
        for i in range(len(courses)):
            newSlot=Exams(courses['code'][i])
            newSlot.room=rnd.randrange(0,len(ROOMS))
            newSlot.time=rnd.randrange(0,len(SLOTS))
            newSlot.teacher=rnd.randrange(0,len(teacher_names))
            self.exams.append(newSlot)
        return self
    
    def calculate_fitness(self):
        weekend=[10,11,12,13]
        self.conflicts=0
        self.fitness=0
        for i in range(len(self.exams)):
            if self.exams[i].time in weekend: # no exam on weekends
                self.conflicts+=1
            for j in range(len(self.exams)):
                if(j>=i):

                    #different exams in same room at the same time
                    if (self.exams[i].time == self.exams[j].time and i!=j and self.exams[i].room==self.exams[j].room): self.conflicts+=1 
                   
                    # checks for teacher
                    if (self.exams[i].teacher == self.exams[j].teacher and i!=j):

                        # one teacher is available at only one slot
                        if(self.exams[i].time==self.exams[j].time):               
                            self.conflicts+=1
                            ##print("\n***",teacher_names[self.exams[i].teacher] , self.exams[j].teacher)

                        # one teacher cant take exams in a row
                        if(SLOTS[self.exams[i].time][:3]==SLOTS[self.exams[j].time][:3] and 
                        SLOTS[self.exams[i].time]!=SLOTS[self.exams[j].time]):          
                            self.conflicts+=1
                            ##print("\n$$",teacher_names[self.exams[i].teacher] , self.exams[j].teacher)
                                
                    if i!=j:
                        y=""
                        # constraints for students
                        if self.exams[i].time==self.exams[j].time:

                            # one exam at one time 

                            for x in range(len(student_dict)):
                                student_dict_list=list(student_dict.values())[x]
                                y=list(student_dict.keys())[x]
                                if (self.exams[i].course in student_dict_list) and (self.exams[j].course in student_dict_list):
                                    self.conflicts+=1
                                    ##print("++",self.exams[i].course,self.exams[j].course,y)
                            # no consecutive exams on one day.
                        elif SLOTS[self.exams[i].time][:3]==SLOTS[self.exams[j].time][:3]:
                            
                            for x in range(len(student_dict)):
                                student_dict_list=list(student_dict.values())[x]
                                y=list(student_dict.keys())[x]
                                if (self.exams[i].course in student_dict_list) and (self.exams[j].course in student_dict_list):
                                    self.conflicts+=1
                                    ##print("--",self.exams[i].course,self.exams[j].course,y)



        self.fitness= (1 / ((self.conflicts)+1))*100 # +1 to save division from 0
        return self.conflicts,self.fitness
        
    #utility function to display timetable.
    def display(self):
        x=PrettyTable()
        x.field_names=["Room","Time","Course Code","Invigilator"]
        for i in range(len(courses)):
            x.add_row([ROOMS[self.exams[i].room],SLOTS[self.exams[i].time],self.exams[i].course,teacher_names[self.exams[i].teacher]])
        print(x)

    
#for i in range(len(student_dict)):
    #print("\n",student_names[i],student_dict[student_names[i]])


In [3]:
def bubblesort(list):
# Swap the elements to arrange in order
    for iter_num in range(len(list)-1,0,-1):
        for idx in range(iter_num):
            if list[idx].fitness<list[idx+1].fitness:
                temp = list[idx]
                list[idx] = list[idx+1]
                list[idx+1] = temp


def generate_population(size):
    population=[]
    for i in range(0,size): population.append(TimeTable().initialize())
    return population

def roullete_wheel_selection(chromosomes):
    max = sum(chromosome.fitness for chromosome in chromosomes)
    pick = random.uniform(0, max)
    current = 0
    for chromosome in chromosomes:
        current += chromosome.fitness
        if current > pick:
            return chromosome

def crossover_population(pop):
    crossover_pop=[]
    for i in range(TOP):
        crossover_pop.append(pop[i])
    crossover_pop.append(cross(pop[0],pop[1]))
    i=TOP+1
    while i< POPULATION_SIZE:
        parent1=roullete_wheel_selection(pop)
        parent2=roullete_wheel_selection(pop)
        #while parent2==parent1:
            #parent2=roullete_wheel_selection(pop)
        #print("\np1,p2",parent1.conflicts,parent2.conflicts)
        crossover_pop.append(cross(parent1,parent2))
        i+=1
    return crossover_pop
def cross(parent1,parent2):
    crossed=TimeTable().initialize()
    for i in range(len(crossed.exams)):
        if(rnd.random() > 0.75): crossed.exams[i]=parent1.exams[i]
        else: crossed.exams[i]=parent2.exams[i]
    return crossed


def mutate_population(mutate_pop):
    #weekend=[10,11,12,13,24,25,26,27]
    
    for i in range(TOP, POPULATION_SIZE): mutation(mutate_pop[i])
        
            #for j in range(len(exams)):
                #if mutate.exams[j].time in weekend: 
                    #mutate.exams[j].time=rnd.randrange(0,len(SLOTS))
            
            
    return mutate_pop       

def mutation(pop):

    mutate=TimeTable().initialize()
    for i in range(len(mutate.exams)):
        if(mutation_probability > rnd.random()): pop.exams[i]=mutate.exams[i]

#population=generate_population(POPULATION_SIZE)
#population=crossover_pop(population)
#population[0].display()


In [4]:
#### debugging utility to display {course:[students enrolled]} ###
#inverse=defaultdict(list)
#for i in range(len(student_course)):
#    inverse[student_course['CourseCode'][i]].append(student_course['StudentName'][i])
#for i in range(len(inverse)):
#    print("\n\n",list(inverse.keys())[i],list(inverse.values())[i])



In [7]:
def runGA():
   
    population = generate_population(POPULATION_SIZE)
    
    print("maximum generations:",max_generations)

    value=[]
    best_solution = None

    for x in range(max_generations):
        p=PrettyTable()
        p.field_names=["Table","conflicts","fitness"]
        ## Calculate Fitness of initial population
        for i in range(POPULATION_SIZE):
            a,f=population[i].calculate_fitness()
        
        bubblesort(population)
        
        for i in range(POPULATION_SIZE):
            p.add_row([i,population[i].conflicts,population[i].fitness])
        
        
        if(x%20==0):
            print("\n\n> Generation",x,"\n")
            print(p)
                
        
        
        ## Selects parents and performs crossover
        new_generation = crossover_population(population)
        

        ## Apply mutation
        mutated = mutate_population(new_generation)

        ## Calculate Fitness of population
        
        population=mutated

        for i in range(POPULATION_SIZE):
            a,f=population[i].calculate_fitness()
        
        bubblesort(population)

        ## Find fittest cadidates
        candidate = population[0]
        if best_solution is None:
            best_solution = candidate
        elif candidate.fitness > best_solution.fitness:
            best_solution = candidate

        #value.append(best_solution)
        if (x+1) == max_generations:
            print(population[0].fitness)
        
        if(x%20==0):
            print ('\nCurrent generation..: {}'.format(x))
            print ('Best solution so far:\n' )
            best_solution.display() 

In [6]:
if __name__ == "__main__":
    max_generations = rnd.randrange(100, 400)
    crossover_probability = round(random.uniform(low=0.3, high=0.75),1)
    mutation_probability = round(random.uniform(low=0.0, high=0.5),1)
    print(mutation_probability)
    runGA()

0.1
maximum generations: 302


> Generation 0 

+-------+-----------+--------------------+
| Table | conflicts |      fitness       |
+-------+-----------+--------------------+
|   0   |    131    | 0.7575757575757576 |
|   1   |    138    | 0.7194244604316548 |
|   2   |    142    | 0.6993006993006993 |
|   3   |    155    | 0.641025641025641  |
|   4   |    160    | 0.6211180124223602 |
|   5   |    165    | 0.6024096385542169 |
|   6   |    187    | 0.5319148936170213 |
|   7   |    200    | 0.4975124378109453 |
|   8   |    206    | 0.4830917874396135 |
|   9   |    222    | 0.4484304932735426 |
+-------+-----------+--------------------+

Current generation..: 0
Best solution so far:

+-------+-----------+-------------+----------------+
|  Room |    Time   | Course Code |  Invigilator   |
+-------+-----------+-------------+----------------+
|  C305 | TH1(9-12) |    CS217    |  Sumera Abbas  |
|  C305 |  T1(2-5)  |    EE227    | Kifayat Ullah  |
|  C304 |  SN1(2-5) |    CS211    |  